# AutoEncoder-去噪

## 宣告套件庫

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## 定義Autoencoder模型

In [2]:
# 定義Autoencoder模型
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12))

        self.decoder = nn.Sequential(
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 28*28),
            nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## 訓練參數設定

In [3]:
# 超參數設定
num_epochs = 10
batch_size = 128
learning_rate = 0.001

# MNIST手寫數字數據集的預處理
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# 下載MNIST數據集並加載訓練集和測試集
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)



# 初始化模型和損失函數
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


100%|██████████| 9912422/9912422 [00:00<00:00, 95657491.63it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 32519649.35it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 25268237.72it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13895352.86it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



## 訓練模型


In [4]:
# 訓練模型
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # 將圖像展平成一維張量
        images = images.reshape(-1, 28*28)

        # 添加噪聲
        noisy_images = images + torch.randn(images.size()) * 0.5

        # 前向傳播和反向傳播
        outputs = model(noisy_images)
        loss = criterion(outputs, images)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 每100個批次打印一次訓練結果
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))







Epoch [1/10], Step [100/469], Loss: 0.2190
Epoch [1/10], Step [200/469], Loss: 0.1597
Epoch [1/10], Step [300/469], Loss: 0.1473
Epoch [1/10], Step [400/469], Loss: 0.1292
Epoch [2/10], Step [100/469], Loss: 0.1077
Epoch [2/10], Step [200/469], Loss: 0.1166
Epoch [2/10], Step [300/469], Loss: 0.1073
Epoch [2/10], Step [400/469], Loss: 0.0997
Epoch [3/10], Step [100/469], Loss: 0.1049
Epoch [3/10], Step [200/469], Loss: 0.1026
Epoch [3/10], Step [300/469], Loss: 0.0864
Epoch [3/10], Step [400/469], Loss: 0.0925
Epoch [4/10], Step [100/469], Loss: 0.0871
Epoch [4/10], Step [200/469], Loss: 0.0913
Epoch [4/10], Step [300/469], Loss: 0.0852
Epoch [4/10], Step [400/469], Loss: 0.0909
Epoch [5/10], Step [100/469], Loss: 0.0855
Epoch [5/10], Step [200/469], Loss: 0.0871
Epoch [5/10], Step [300/469], Loss: 0.0807
Epoch [5/10], Step [400/469], Loss: 0.0827
Epoch [6/10], Step [100/469], Loss: 0.0847
Epoch [6/10], Step [200/469], Loss: 0.0759
Epoch [6/10], Step [300/469], Loss: 0.0743
Epoch [6/10

## 測試模型

In [5]:
# 測試模型
from google.colab.patches import cv2_imshow
import numpy as np
with torch.no_grad():
    for i,(images, _) in enumerate(test_loader):
        images = images.reshape(-1, 28*28)
        noisy_images = images + torch.randn(images.size()) * 0.5
        
        outputs = model(noisy_images)

        outputs=outputs.view(outputs.size(0), 1, 28, 28)
        images=images.view(images.size(0), 1, 28, 28)
        noisy_images=noisy_images.view(noisy_images.size(0), 1, 28, 28)
        print('------')
        print('原始影像')
        cv2_imshow(np.array(images[0][0])*255)
        print('噪音影像')
        cv2_imshow(np.array(noisy_images[0][0])*255)
        print('去噪影象')
        cv2_imshow(np.array(outputs[0][0])*255)
        if i>5:

            break

------
原始影像


噪音影像


去噪影象


------
原始影像


噪音影像


...

噪音影像


去噪影象


------
原始影像


噪音影像


去噪影象
